In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("../data/output.csv")
data.head()
COMMUNE = "yopougon"

In [3]:
data.columns

Index(['Unnamed: 0', 'post_id', 'text', 'post_text', 'shared_text',
       'original_text', 'time', 'timestamp', 'image', 'image_lowquality',
       'images', 'images_description', 'images_lowquality',
       'images_lowquality_description', 'video', 'video_duration_seconds',
       'video_height', 'video_id', 'video_quality', 'video_size_MB',
       'video_thumbnail', 'video_watches', 'video_width', 'likes', 'comments',
       'shares', 'post_url', 'link', 'links', 'user_id', 'username',
       'user_url', 'is_live', 'factcheck', 'shared_post_id', 'shared_time',
       'shared_user_id', 'shared_username', 'shared_post_url', 'available',
       'comments_full', 'reactors', 'w3_fb_url', 'reactions', 'reaction_count',
       'with', 'page_id', 'sharers', 'image_id', 'image_ids', 'was_live',
       'header', 'fetched_time', 'video_ids', 'videos', 'listing_title',
       'listing_price', 'listing_location'],
      dtype='object')

In [5]:
# conserver ques la colonne post_id et post_text listing_price et listing_location
data = data[['post_text','time','post_url']]
data.head()

,post_text,time,post_url
0,‟ Yannick Blehi s’est vu être dépouillé de son...,2023-02-15 09:11:54,https://facebook.com/173550876582729/posts/125...
1,INFO : 0702235030\n\nGRANDE 4PIECES 140.000FR ...,2023-02-18 09:10:42,https://facebook.com/173550876582729/posts/125...
2,INFO : 0702235030\n\nBONNE 2PIECES 40.000FR VI...,2023-02-18 09:38:52,https://facebook.com/173550876582729/posts/125...
3,INFO : 0702235030\n\nRÉSIDENCE MEUBLE HAUT STA...,2023-02-18 09:35:15,https://facebook.com/173550876582729/posts/125...
4,INFO : 0702235030\n\nGRANDE RÉSIDENCE MEUBLE S...,2023-02-18 09:30:41,https://facebook.com/173550876582729/posts/125...


In [6]:
num = 5
print(data.iloc[num,0])
temp = {"post_text":data.iloc[num,0], "time":data.iloc[num,1]}

INFO : 0702235030

CAVE À CÉDÉ : 2.200.000FR A DÉBATTRE VISITE : 5000FR, LOYER : 50.000FR
LIEU : YOPOUGON SICOGIE PMI EN BORDURE
…


In [6]:
# premiere colonne de ma premiere ligne
data.iloc[0,0]

# 2e colonne de ma premiere ligne
data.iloc[0,1]

'2023-02-15 09:11:54'

In [7]:
import spacy

nlp = spacy.load("fr_core_news_sm")


def extract_features(post_text):
    results = {
        "loyer": None,
        "nombre_piece": None,
        "annee_publication": None,
        "jour_publication": None,
        "mois_publication": None,
        "type_logement": None,
        "standing": None,
        "commune": None,
        "quartier": None,
        "nb_loyer": None,
        "lien": None,
    }

    # Conversion post_text en dataframe
    post_df = pd.DataFrame(post_text.apply(pd.Series))

    # Si le post est nan on retourne un dictionnaire vide
    if pd.isna(post_df.iloc[0, 0]):
        return results

    # EXTRACTION LOYER & NOMBRE DE PIECES
    post_text = {"texte": post_df.iloc[0, 0], "date": post_df.iloc[1, 0]}
    doc = nlp(post_text["texte"])
    results["lien"] = post_df.iloc[2, 0]
    results["commune"] = COMMUNE

    try:
        for i, token in enumerate(doc):
            if token.text.endswith("fr") or token.text.endswith("f") or token.text.endswith("0"):
                results["loyer"] = token.text
                break
            elif token.text.lower() == "loyer":
                next_token = doc[i + 1] if i + 1 < len(doc) else None
                while next_token and not next_token.like_num:
                    i += 1
                    next_token = doc[i + 1] if i + 1 < len(doc) else None
                if next_token and next_token.like_num:
                    results["loyer"] = next_token.text
                    break
    except:
        pass

    try:
        l = [t for t in doc if 'pièces' in t.text.lower() and t.pos_ == "NOUN"]

        if l:
            term = l[0]
            subtree = [t for t in term.subtree]
            pieces = [t for t in subtree if t.text.lower() == "pièces" or t.text.lower() == "fr cfa" or t.text.lower() == "franc cfa" or t.text.lower() == "francs cfa"]
            if pieces:
                nb_piece = pieces[0].nbor(-1)
                if nb_piece.like_num == True:
                    results["nombre_piece"] = nb_piece.text
    except:
        pass

    # EXTRACTION ANNEE, MOIS, JOUR
    try:
        annee, mois, jour = post_text["date"].split(" ")[0].split("-")
        results["annee_publication"] = annee
        results["mois_publication"] = mois
        results["jour_publication"] = jour
    except:
        pass

    try:
        for i, token in enumerate(doc):
            if token.text.lower() == "immeuble" or token.text.lower() == "étage":
                results["type_logement"] = "immeuble"
                break
            elif token.text.lower() == "maison":
                results["type_logement"] = "appartement"
                break
    except:
        pass

    return results


In [8]:
# appliquer la fonction extract_features sur chaque ligne du dataframe data et creer un nouveau dataframe avec les resultats
data_features = data.apply(extract_features, axis=1)
data_features = pd.DataFrame(data_features.apply(pd.Series))
data_features.shape

(167, 11)

In [9]:
data_features.to_csv("../data/resultat.csv", index=False)
data_features.head(50)

,loyer,nombre_piece,annee_publication,jour_publication,mois_publication,type_logement,standing,commune,quartier,nb_loyer,lien
0,objectif,02,2023,15,02,immeuble,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
1,0702235030,None,2023,18,02,immeuble,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
2,0702235030,None,2023,18,02,None,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
3,0702235030,None,2023,18,02,None,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
4,0702235030,None,2023,18,02,immeuble,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
5,0702235030,None,2023,18,02,None,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
6,0702235030,None,2023,18,02,None,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
7,0702235030,None,2023,18,02,immeuble,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
8,0702235030,None,2023,18,02,immeuble,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
9,0702235030,None,2023,18,02,None,None,yopougon,None,None,https://facebook.com/173550876582729/posts/125...
